# Lab | Customer Analysis Round 6

In [1]:

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant


file_path = r"C:\Users\ashis\Downloads\marketing_customer_analysis.csv"

data = pd.read_csv(file_path)

# X-y split.

# Display the first few rows of the dataframe to understand its structure
data.head()

# Define the target variable
y = data['Customer Lifetime Value']

# Define the feature variables, excluding the target and non-predictive columns
X = data.drop(columns=['Customer Lifetime Value', 'Customer', 'Unnamed: 0'])

# Display the shapes of X and y to confirm the split
X.shape, y.shape



((10910, 23), (10910,))

In [2]:
# Normalize (numerical).

from sklearn.preprocessing import StandardScaler

# Identify numerical columns
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns

# Initialize the scaler
scaler = StandardScaler()

# Fit and transform the numerical columns
X[numerical_cols] = scaler.fit_transform(X[numerical_cols])

# Display the first few rows of the normalized data
X.head()


,State,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,Income,Location Code,Marital Status,...,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,Vehicle Type
0,Arizona,No,Basic,College,2/18/11,Employed,M,0.345635,Suburban,Married,...,-0.421143,2.509455,Corporate Auto,Corporate L3,Offer3,Agent,-0.486325,Four-Door Car,Medsize,NaN
1,California,No,Basic,College,1/18/11,Unemployed,F,-1.236462,Suburban,Single,...,-0.421143,-0.824922,Personal Auto,Personal L3,Offer4,Call Center,1.061160,Four-Door Car,Medsize,NaN
2,Washington,No,Basic,Bachelor,2/10/11,Employed,M,-0.507194,Suburban,Single,...,-0.421143,-0.408125,Personal Auto,Personal L3,Offer3,Call Center,0.154404,SUV,Medsize,A
3,Oregon,Yes,Extended,College,1/11/11,Employed,M,0.380189,Suburban,Single,...,-0.421143,-0.408125,Corporate Auto,Corporate L3,Offer2,Branch,0.168140,Four-Door Car,Medsize,A
4,Oregon,No,Premium,Bachelor,1/17/11,Medical Leave,F,-0.456597,Suburban,Married,...,NaN,1.675860,Personal Auto,Personal L2,Offer1,Branch,0.934524,Four-Door Car,Medsize,NaN


In [3]:
# One Hot/Label Encoding (categorical).

# Identify categorical columns
categorical_cols = X.select_dtypes(include=['object']).columns

# Apply one-hot encoding
X_encoded = pd.get_dummies(X, columns=categorical_cols, drop_first=True)

# Display the first few rows of the encoded data
X_encoded.head()


# Identify categorical columns
categorical_cols = X.select_dtypes(include=['object']).columns

# Apply one-hot encoding
X_encoded = pd.get_dummies(X, columns=categorical_cols, drop_first=True)

# Display the first few rows of the encoded data
X_encoded.head()


,Income,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Total Claim Amount,State_California,State_Nevada,State_Oregon,...,Sales Channel_Branch,Sales Channel_Call Center,Sales Channel_Web,Vehicle Class_Luxury Car,Vehicle Class_Luxury SUV,Vehicle Class_SUV,Vehicle Class_Sports Car,Vehicle Class_Two-Door Car,Vehicle Size_Medsize,Vehicle Size_Small
0,0.345635,-0.934819,-0.808451,0.139877,-0.421143,2.509455,-0.486325,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,-1.236462,-0.847713,-1.205282,-0.790709,-0.421143,-0.824922,1.061160,1,0,0,...,0,1,0,0,0,0,0,0,1,0
2,-0.507194,0.197554,1.870158,-0.611750,-0.421143,-0.408125,0.154404,0,0,0,...,0,1,0,0,0,1,0,0,1,0
3,0.380189,0.110448,-0.510828,-1.613919,-0.421143,-0.408125,0.168140,0,0,1,...,1,0,0,0,0,0,0,0,1,0
4,-0.456597,0.691152,NaN,-0.611750,NaN,1.675860,0.934524,0,0,1,...,1,0,0,0,0,0,0,0,1,0


In [4]:
# Concat DataFrames

# Assuming X_encoded and y are already defined and preprocessed

# Concatenate the feature DataFrame and the target variable
data_final = pd.concat([X_encoded, y], axis=1)

# Display the first few rows of the concatenated DataFrame
data_final.head()



,Income,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Total Claim Amount,State_California,State_Nevada,State_Oregon,...,Sales Channel_Call Center,Sales Channel_Web,Vehicle Class_Luxury Car,Vehicle Class_Luxury SUV,Vehicle Class_SUV,Vehicle Class_Sports Car,Vehicle Class_Two-Door Car,Vehicle Size_Medsize,Vehicle Size_Small,Customer Lifetime Value
0,0.345635,-0.934819,-0.808451,0.139877,-0.421143,2.509455,-0.486325,0,0,0,...,0,0,0,0,0,0,0,1,0,4809.216960
1,-1.236462,-0.847713,-1.205282,-0.790709,-0.421143,-0.824922,1.061160,1,0,0,...,1,0,0,0,0,0,0,1,0,2228.525238
2,-0.507194,0.197554,1.870158,-0.611750,-0.421143,-0.408125,0.154404,0,0,0,...,1,0,0,0,1,0,0,1,0,14947.917300
3,0.380189,0.110448,-0.510828,-1.613919,-0.421143,-0.408125,0.168140,0,0,1,...,0,0,0,0,0,0,0,1,0,22332.439460
4,-0.456597,0.691152,NaN,-0.611750,NaN,1.675860,0.934524,0,0,1,...,0,0,0,0,0,0,0,1,0,9025.067525


# Linear Regression

In [6]:
# Train-test split.
# Apply linear regression.

# Model Validation
# Description:
# R2.
# MSE.
# RMSE.
# MAE

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Define the target variable
target = 'Total Claim Amount'

# Select relevant features
features = ['Customer Lifetime Value', 'Response', 'Coverage', 'Education', 'EmploymentStatus', 
            'Gender', 'Income', 'Marital Status', 'Monthly Premium Auto', 'Number of Open Complaints', 
            'Number of Policies', 'Policy Type', 'Renew Offer Type', 'Sales Channel', 'Vehicle Class', 
            'Vehicle Size']

# Extract the feature matrix (X) and target vector (y)
X = data[features]
y = data[target]

# Handle missing values by filling them with the median for numerical columns and the most frequent for categorical
for col in X.select_dtypes(include=['float64', 'int64']).columns:
    X[col].fillna(X[col].median(), inplace=True)

for col in X.select_dtypes(include=['object']).columns:
    X[col].fillna(X[col].value_counts().idxmax(), inplace=True)

# Encode categorical variables using OneHotEncoder
categorical_features = X.select_dtypes(include=['object']).columns.tolist()
numerical_features = X.select_dtypes(exclude=['object']).columns.tolist()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numerical_features),
        ('cat', OneHotEncoder(drop='first'), categorical_features)
    ])

# Create a pipeline with preprocessor and linear regression model
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)


print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R²) score: {r2}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"Mean Absolute Error (MAE): {mae}")

C:\Users\ashis\AppData\Local\Temp\ipykernel_2664\3966272997.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col].fillna(X[col].median(), inplace=True)
C:\Users\ashis\AppData\Local\Temp\ipykernel_2664\3966272997.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col].fillna(X[col].value_counts().idxmax(), inplace=True)
C:\Users\ashis\AppData\Local\Temp\ipykernel_2664\3966272997.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-co

Mean Squared Error (MSE): 34574.34273697968
R-squared (R²) score: 0.5717975120822207
Root Mean Squared Error (RMSE): 185.94177243691016
Mean Absolute Error (MAE): 137.134429450512
